In [1]:
import numpy as np
import scipy as sp

import matplotlib.pyplot as plt
import matplotlib.axes as axe
import pandas as pd
import datetime as dt
import gurobipy as gp
from gurobipy import GRB
import cvxpy as cp

import random
from itertools import chain, combinations, tee
import time

plt.rcParams['text.usetex'] = True

# Functions

In [2]:
def demand_name_by_group_index(index):
    list_demand_names = ["Demand (eligible group, 1)", "Demand (eligible group, 2)", \
                         "Demand (ineligible group, 1)", "Demand (ineligible group, 2)", \
                         "Demand (ineligible group, 3)"]
    return list_demand_names[index]

def VoT_name_by_group_index(index):
    list_demand_names = ["VoT (eligible group, 1)", "VoT (eligible group, 2)", \
                         "VoT (ineligible group, 1)", "VoT (ineligible group, 2)", \
                         "VoT (ineligible group, 3)"]
    return list_demand_names[index]


# Groups, Routes to Edges:

In [3]:
directory_path = '../data/data_income_percentage_VoT___101_N_Sep_to_Nov_2024/'
df_data = pd.read_csv(directory_path + 'data_cities_od_VoTs_demands_1.csv')

# df_od_flow_data
# df_data

In [4]:
dict_data = {}

for column_name_full in list(df_data.columns):
    if column_name_full == "Data Category":
        categories_list = df_data[column_name_full].tolist()
    else:
        dict_data[int(column_name_full)] = {}
        for category_index, category in enumerate(categories_list):
            if category == "Start City Index" or category == "End City Index":
                dict_data[int(column_name_full)][category] \
                    = int(df_data[column_name_full].tolist()[category_index])
            elif category == "Start City" or category == "End City":
                dict_data[int(column_name_full)][category] \
                    = df_data[column_name_full].tolist()[category_index]
            else:
#                 print("category:", category)
                dict_data[int(column_name_full)][category] \
                    = float(df_data[column_name_full].tolist()[category_index])


In [5]:
dict_data

{0: {'Start City Index': 0,
  'End City Index': 0,
  'Start City': 'Palo Alto',
  'End City': 'Palo Alto',
  'O-D Flow (Max Entropy)': 612.539616936298,
  'Demand (eligible group, 1)': 47.77809012103125,
  'VoT (eligible group, 1)': 0.03197423570019724,
  'Demand (eligible group, 2)': 27.564282762133406,
  'VoT (eligible group, 2)': 0.10238603988603988,
  'Demand (ineligible group, 1)': 115.15744798402402,
  'VoT (ineligible group, 1)': 0.2754407051282051,
  'Demand (ineligible group, 2)': 134.75871572598555,
  'VoT (ineligible group, 2)': 0.5809294871794872,
  'Demand (ineligible group, 3)': 287.28108034312373,
  'VoT (ineligible group, 3)': 1.8596449415012848},
 1: {'Start City Index': 0,
  'End City Index': 1,
  'Start City': 'Palo Alto',
  'End City': 'East Palo Alto',
  'O-D Flow (Max Entropy)': 98.03491986336807,
  'Demand (eligible group, 1)': 7.64672374934271,
  'VoT (eligible group, 1)': 0.03197423570019724,
  'Demand (eligible group, 2)': 4.411571393851563,
  'VoT (eligible g

In [6]:
cities_dict = {}
for od_info in list(dict_data.values()):
    if od_info["Start City Index"] not in list(cities_dict.keys()):
        cities_dict[od_info["Start City Index"]] = od_info["Start City"]
    if od_info["End City Index"] not in list(cities_dict.keys()):
        cities_dict[od_info["End City Index"]] = od_info["End City"]

cities_list = list(cities_dict.values())

# cities_dict

In [7]:
od_to_edges_array = np.zeros((len(list(dict_data.keys())), 2))

for od_index, od_info in dict_data.items():
    od_to_edges_array[od_index, 0] = int(cities_list.index(od_info["Start City"]))
    od_to_edges_array[od_index, 1] = int(cities_list.index(od_info["End City"]))

# od_to_edges_array

In [8]:
num_groups_per_od = 5

demand_array = np.zeros((len(list(dict_data.keys())), num_groups_per_od))
VoT_array_base = np.zeros((len(list(dict_data.keys())), num_groups_per_od))

for od_index, od_value in dict_data.items():
    for group_index in range(num_groups_per_od):
        demand_name = demand_name_by_group_index(group_index)
        VoT_name = VoT_name_by_group_index(group_index)
        
        demand_array[od_index, group_index] = od_value[demand_name]
        VoT_array_base[od_index, group_index] = od_value[VoT_name]

# print(demand_array)
# VoT_array_base

In [9]:
directory_path = '../data/data_income_percentage_VoT___101_N_Sep_to_Nov_2024/'

VoT_array = np.zeros((VoT_array_base.shape[0], VoT_array_base.shape[1], 5))

T = 5
for t in range(T):
    df_perturbation_data = pd.read_csv(directory_path + 'perturbations_1_' + str(t) + '.csv')
    perturbation_array = df_perturbation_data.to_numpy()[:, 1:]
    VoT_array[:, :, t] = VoT_array_base * perturbation_array
    
# VoT_array_base
# perturbation_array

# Download Latency Parameters Data

In [10]:
directory_path_latency = '../data/pems_latency_inference___101_N_Sep_to_Nov_2024/'
df_latency_params = pd.read_csv(directory_path_latency + 'latency_params.csv')

# list(df_latency_params.loc[:, "Palo Alto"])

In [11]:
dict_latency_params = {}

city_list = list(df_latency_params.columns)[1:]

for city in city_list:
#     if city != "Belmont":
    if 1 == 1:
        dict_latency_params[city] = {}
        dict_latency_params[city]["Flow (at bend)"] = df_latency_params.loc[:, city][0]
        dict_latency_params[city]["Latency (at bend)"] = df_latency_params.loc[:, city][1]
        dict_latency_params[city]["Slope (after bend)"] = df_latency_params.loc[:, city][2]

dict_latency_params

{'Palo Alto': {'Flow (at bend)': 861.9885,
  'Latency (at bend)': 1.326448252,
  'Slope (after bend)': 0.000782666},
 'East Palo Alto': {'Flow (at bend)': 1001.517857,
  'Latency (at bend)': 2.213126553,
  'Slope (after bend)': 0.000584484},
 'Redwood City': {'Flow (at bend)': 881.1846667,
  'Latency (at bend)': 4.892192375,
  'Slope (after bend)': 0.001563724},
 'Belmont': {'Flow (at bend)': 1278.948125,
  'Latency (at bend)': 1.199911179,
  'Slope (after bend)': 0.001994138},
 'San Mateo': {'Flow (at bend)': 1034.092826,
  'Latency (at bend)': 5.541006284,
  'Slope (after bend)': 0.002147262},
 'Burlingame': {'Flow (at bend)': 845.15,
  'Latency (at bend)': 1.503111345,
  'Slope (after bend)': 0.000306601},
 'Millbrae': {'Flow (at bend)': 853.1818182,
  'Latency (at bend)': 2.384328452,
  'Slope (after bend)': 0.000321856}}

# General CBCP Equilibrium Solver

## (Special Case) Quartic Polynomial Latency Functions

In [12]:
# grad = np.array([0, 1, 2, 3, 4])
grad = np.array([2, 4, 0, 1, 3])

for id_temp, entry_temp in enumerate(grad):
    print("id_temp, entry_temp:", id_temp, entry_temp)


id_temp, entry_temp: 0 2
id_temp, entry_temp: 1 4
id_temp, entry_temp: 2 0
id_temp, entry_temp: 3 1
id_temp, entry_temp: 4 3


In [13]:
# np.array([1, 2, 3]) * np.array([6, 5, 3])

In [14]:
def latency_max(flow_max, coeff):
    
    assert np.all(coeff >= 0.0), "coeff should be non-negative"
    assert len(coeff.shape) == 1, "coeff should be a 1-D array."
    assert coeff.shape[0] == 3, "Latency functions are assumed to be piecewise linear / affine with 3 parameters."
    
    return coeff[0] + max(coeff[1] * (flow_max - coeff[2]), 0)

def welfare_obj(T, num_edges, num_gp_lanes, lambda_E, lambda_R, lambda_I, tau, alpha, \
                demand_array, VoT_array, num_el, od_to_edges_array, y, \
                coeff_input):

#     coeff = coeff_from_input(coeff_input, num_edges, num_gp_lanes)

    assert np.all((alpha >= -1E-3) & (alpha <= 1.001)), "We must have alpha in [0, 1] component-wise!"
    
    coeff = coeff_input
    
    assert len(od_to_edges_array.shape) == 2, "od_to_edges should be 2-dimensional."
    assert od_to_edges_array.shape[1] == 2, "od_to_edges second dimension should be for start and end edges."
    
    edge_to_od_dict = {}
    for e in range(num_edges):
        edge_to_od_dict[e] = [k for k in list(range(int(od_to_edges_array.shape[0]) )) \
                               if od_to_edges_array[k, 0] <= e <= od_to_edges_array[k, 1]]

    num_groups = demand_array.shape[1]
    num_in = num_groups - num_el
    assert num_in >= 0, "We must have num_in >= 0."
    
    el_indices = list(range(num_el))
    in_indices = list(range(num_el, num_groups))
    
#     print()
#     print("tau.shape[0]:", tau.shape[0])
#     print("num_edges:", num_edges)
#     print()

    assert len(tau.shape) == 2, "tau should be 2-dimensional."
    assert tau.shape[0] == num_edges, "toll vector's first axis length must equal the number of edges."
    assert tau.shape[1] == T, "toll vector's second axis length must equal the time horizon."
    
    ## Compute lane flows:
    
    x = np.zeros((num_edges, 2, T))
    for e in range(num_edges):
        for t in range(T):
            x[e, 0, t] += sum(y[(od, g, e, 0, t)] for od in edge_to_od_dict[e] \
                              for g in range(num_groups))
    for e in range(num_edges):
        for t in range(T):
            x[e, 1, t] += sum( (y[(od, g, e, 0, t)] + y[(od, g, e, 1, t)]) for od in edge_to_od_dict[e] \
                              for g in el_indices)
            x[e, 1, t] += sum(y[(od, g, e, 1, t)] for od in edge_to_od_dict[e] \
                              for g in in_indices)
            
    print()
    print("in_indices:", in_indices)
    print("el_indices:", el_indices)
    print()
    
    ## Compute lane latencies:
    
    ell = np.zeros((num_edges, 2, T))
    for e in range(num_edges):
        for t in range(T):
            ell[e, 0, t] = coeff[0, e] + coeff[1, e] * max(x[e, 0, t] - coeff[2, e], 0)
            ell[e, 1, t] = coeff[0, e] + coeff[1, e] * max(x[e, 1, t]/num_gp_lanes - coeff[2, e], 0)

    obj_E = sum( y[(od, g, e, 0, t)] * (VoT_array[od, g, t] * ell[e, 0, t] + (1 - alpha[e, t]) * tau[e, t] ) \
                for e in range(num_edges) for od in edge_to_od_dict[e] \
                for g in el_indices for t in range(T) ) \
            + sum( y[(od, g, e, 1, t)] * (VoT_array[od, g, t] * ell[e, 0, t] + tau[e, t]) \
                for e in range(num_edges) for od in edge_to_od_dict[e] \
                for g in el_indices for t in range(T) ) \
            + sum( y[(od, g, e, 2, t)] * VoT_array[od, g, t] * ell[e, 1, t] \
                  for e in range(num_edges) for od in edge_to_od_dict[e] \
                  for g in el_indices for t in range(T) )
    obj_I = sum( y[(od, g, e, 0, t)] * (VoT_array[od, g, t] * ell[e, 0, t] + tau[e, t]) \
                for e in range(num_edges) for od in edge_to_od_dict[e] \
                for g in in_indices for t in range(T) ) \
            + sum( y[(od, g, e, 1, t)] * VoT_array[od, g, t] * ell[e, 1, t] \
                for e in range(num_edges) for od in edge_to_od_dict[e] \
                  for g in in_indices for t in range(T) )
    obj_R = sum( y[(od, g, e, 0, t)] * (1 - alpha[e, t]) * tau[e, t] \
                for e in range(num_edges) for od in edge_to_od_dict[e] \
                for g in el_indices for t in range(T) ) \
            + sum( y[(od, g, e, 0, t)] * tau[e, t] \
                for e in range(num_edges) for od in edge_to_od_dict[e] \
                for g in in_indices for t in range(T) )

    welfare = lambda_E * obj_E - lambda_R * obj_R + lambda_I * obj_I
    
        

#     print()
#     print("obj_E:", obj_E)
#     print("obj_R:", obj_R)
#     print("obj_I:", obj_I)
#     print("welfare:", welfare)
#     print()
    
    return welfare, obj_E, obj_R, obj_I


# Latency, total, throughout the entire time horizon. 

def latency_total(T, num_edges, lambda_E, lambda_R, lambda_I, tau, \
                  demand_array, VoT_array, num_el, y, od_to_edges_array, \
                  coeff_input):
    
#     coeff = coeff_from_input(coeff_input, num_edges, num_gp_lanes)

    coeff = coeff_input
    
    assert len(od_to_edges_array.shape) == 2, "od_to_edges should be 2-dimensional."
    assert od_to_edges_array.shape[1] == 2, "od_to_edges' second dimension should be for start and end edges."
    
    edge_to_od_dict = {}
    for edge in range(num_edges):
        edge_to_od_dict[e] = [k for k in list(range(int(od_to_edges_array.shape[0]) )) \
                               if od_to_edges_array[k, 0] <= e <= od_to_edges_array[k, 1]]
    
    # In full:
    # y indices: (od, group, edge, "lane", time)

    num_groups = demand_array.shape[1]
    num_in = num_groups - num_el
    assert num_in >= 0, "We must have num_in >= 0."
    
    el_indices = list(range(num_el))
    in_indices = list(range(num_el, num_groups))
    
#     print()
#     print("tau.shape[0]:", tau.shape[0])
#     print("num_edges:", num_edges)
#     print()

    assert len(tau.shape) == 2, "tau should be 2-dimensional."
    assert tau.shape[0] == num_edges, "toll vector's first axis length must equal the number of edges."
    assert tau.shape[1] == T, "toll vector's second axis length must equal the time horizon."
    
    ## Compute lane flows:
    
    x = np.zeros((num_edges, 2, T))
    for e in range(num_edges):
        for t in range(T):
            x[e, 0, t] += sum(y[(od, g, e, 0, t)] for od in edge_to_od_dict[e] \
                              for g in range(num_groups))
    for e in range(num_edges):
        for t in range(T):
            x[e, 1, t] += sum( (y[(od, g, e, 0, t)] + y[(od, g, e, 1, t)]) for od in edge_to_od_dict[e] \
                              for g in el_indices)
            x[e, 1, t] += sum(y[(od, g, e, 1, t)] for od in edge_to_od_dict[e] \
                              for g in in_indices)
    
    ## Compute lane latencies:
    
    ell = np.zeros((num_edges, 2, T))
    for e in range(num_edges):
        for t in range(T):
            ell[e, 0, t] = coeff[0, e] + coeff[1, e] * max(x[e, 0, t] - coeff[2, e], 0)
            ell[e, 1, t] = coeff[0, e] + coeff[1, e] * max(x[e, 1, t]/num_gp_lanes - coeff[2, e], 0)

    latency = sum( x[e, k, t] * ell[e, k, t] \
                for e in range(num_edges) for k in range(2) for t in range(T) )

#     print()
#     print("obj_E:", obj_E)
#     print("obj_R:", obj_R)
#     print("obj_I:", obj_I)
#     print("welfare:", welfare)
#     print()
    
    return latency

In [15]:
# arr = np.arange(5)
# arr.reshape((5, 1))

In [16]:
def proj_tau_alpha(T, num_edges, tau, alpha, od_to_edges_list_full, tau_max):

#     print()
#     print("tau.shape[0]:", tau.shape[0])
#     print("num_edges:", num_edges)
#     print()

    assert tau.shape[0] == num_edges, "tau must have length equal to the number of edges."
    assert tau_max.shape[0] == num_edges, "tau_max must have length equal to the number of edges."
    
    tau_feas = cp.Variable((num_edges, T))
    
    func = cp.sum_squares(tau_feas - tau)

    objective = cp.Minimize(func)

    constraints = []
    constraints += [tau_feas >= 0.0]
    constraints += [tau_feas <= tau_max]
    
    prob = cp.Problem(objective, constraints)
    result = prob.solve()
    
    alpha_feas = np.clip(alpha, 0.0, 1.0)

    print()
    print("tau_feas.value:", np.round(np.maximum(tau_feas.value, 0.0), decimals=3))
    print()
    print("alpha_feas:", alpha_feas)
    print()
    
#     print("alpha:", alpha)
#     print()

    return np.round(np.maximum(tau_feas.value, 0.0), decimals=3), \
            np.round(np.maximum(alpha_feas, 0.0), decimals=3)
    
#     return tau_feas.value, alpha_feas



# Chinmay's Algorithm:

## Convex Program for CBCP and DBCP Equilibria:

In [17]:
## Steps
# 1: Define variables
# 2: Define objective
# 3: Define constraints
# 4: Define problem
# 5: Solve problem
# 6: Extract values

In [18]:
# num_edges = 8
# od_to_edges_array = np.array([[0, 0], [0, 3], [0, 7], [1, 5], [1, 7], [3, 3], [3, 6]])

# od_to_edges_list_full = [list(range(od_to_edges_array[od, 0], od_to_edges_array[od, 1] + 1)) \
#                       for od in range(od_to_edges_array.shape[0])]

# edge_to_od = [[od for od in range(od_to_edges_array.shape[0]) if e in od_to_edges_list_full[od]] \
#             for e in range(num_edges)]

# print(od_to_edges_list_full)
# print()
# print(edge_to_od)

In [19]:

# Below: For quartic latency functions:
# Latency Function: a_4 x^4 + a_3 x^3 + a_2 x^2 + a_1 x + a_0

def solve_DBCP_direct(T, num_edges, num_gp_lanes, tau, alpha, od_to_edges_array, \
                      demand_array, VoT_array, num_el, coeff_input):
    
    assert tau.shape == alpha.shape, "We must have tau.shape == alpha.shape!"
    assert np.all((alpha >= -1E-3) & (alpha <= 1.001))
    
    alpha = np.clip(alpha, 0.0, 1.0)
    
#     print("tau (in solve_DBCP_direct):", tau)
    
#     coeff = coeff_from_input(coeff_input, num_edges, num_gp_lanes)

    coeff = coeff_input
    
    assert VoT_array.shape[2] == T, "We should have VoT_array.shape[2] == T."
    assert demand_array.shape == VoT_array[:, :, 0].shape, "demand_array and VoT_array[:, :, 0] should have identical shape."
    assert np.all(demand_array > 0.0), "Each entry of demand_array must be strictly positive."
    assert np.all(tau >= 0.0), "Each entry of tau must be non-negative."
    assert num_el <= demand_array.shape[1], "num_el, the number of eligible income groups, should not exceed \
                                            demand_array.shape[1], which should equal the number of income groups."
    
    num_groups = demand_array.shape[1]
    
    ## Variable indices:
    # y indices: (od, income group, edge, "lane", time)
    # y indices: (od, income group, edge, "lane", time)

    el_indices = list(range(num_el))
    in_indices = list(range(num_el, num_groups))
    group_indices = list(range(num_groups))
    
    od_to_edges_list_full = [list(range(int(od_to_edges_array[od, 0]), int(od_to_edges_array[od, 1]) + 1)) \
                          for od in range(od_to_edges_array.shape[0])]
    
    edge_to_ods = [[od for od in range(od_to_edges_array.shape[0]) if e in od_to_edges_list_full[od]] \
                   for e in range(num_edges)]
    
    num_od = len(od_to_edges_list_full)
    
    # Variables:
    y = {}
    for od in range(num_od):
        for e in od_to_edges_list_full[od]:
            for t in range(T):
                for g in el_indices:
                    for k in [0, 1, 2]:
                        y[(od, g, e, k, t)] = cp.Variable(1)
                for g in in_indices:
                    for k in [0, 1]:
                        y[(od, g, e, k, t)] = cp.Variable(1)
    
    x = {}
    for e in range(num_edges):
        for k in [0, 1]:
            for t in range(T):
                x[(e, k, t)] = cp.Variable(1)

    # Objective:
    func = 0.0
    for e in range(num_edges):
        for t in range(T):
            func += coeff[0, e] * x[e, 0, t] \
                    + 0.5 * coeff[1, e] * cp.square(cp.maximum(x[e, 0, t] - coeff[2, e], 0))
            func += coeff[0, e] * x[e, 1, t] \
                    + 0.5 * coeff[1, e] * cp.square(cp.maximum(x[e, 1, t]/num_gp_lanes - coeff[2, e], 0))
    
#     print("VoT_array.shape:", VoT_array.shape)
#     print("el_indices:", el_indices)
#     print("in_indices:", in_indices)
    
    for od in range(num_od):
        for e in od_to_edges_list_full[od]:
            for t in range(T):
                for g in el_indices:
                    func += (1 - alpha[e, t]) * tau[e, t] * y[(od, g, e, 0, t)] / VoT_array[od, g, t]
                    func += tau[e, t] * y[(od, g, e, 1, t)] / VoT_array[od, g, t]
                for g in in_indices:
                    func += tau[e, t] * y[(od, g, e, 0, t)] / VoT_array[od, g, t]

    objective = cp.Minimize(func)
    
    # Constraints:
    constraints = []
    
    constraints += [y[(od, g, e, k, t)] >= 0.0 for od in range(num_od) \
                    for e in od_to_edges_list_full[od] for g in el_indices  \
                    for k in [0, 1, 2] for t in range(T)]
    constraints += [y[(od, g, e, k, t)] >= 0.0 for od in range(num_od) \
                    for e in od_to_edges_list_full[od] for g in in_indices  \
                    for k in [0, 1] for t in range(T)]
    constraints += [x[(e, k, t)] >= 0.0 for e in range(num_edges) \
                    for k in [0, 1] for t in range(T)]
    
#     print()
#     print("demand_array:", demand_array)
#     print()
#     print("alpha:", alpha)
#     print()
#     print("el_indices:", el_indices)
#     print()
#     print("in_indices:", in_indices)
#     print()
#     print("edge_to_ods:", edge_to_ods)
#     print()
#     print("od_to_edges_list_full:", od_to_edges_list_full)
#     print()
    
    od_g_e_t_list = []

    for e in range(num_edges):
        for t in range(T):
            
            ## Edge contraints:
            constraints += [sum( y[(od, g, e, 0, t)] + y[(od, g, e, 1, t)] for od in edge_to_ods[e] for g in el_indices) \
                                + sum( y[(od, g, e, 0, t)] for od in edge_to_ods[e] for g in in_indices ) \
                                == x[(e, 0, t)] ]
            constraints += [sum( y[(od, g, e, 2, t)] for od in edge_to_ods[e] for g in el_indices) \
                                + sum( y[(od, g, e, 1, t)] for od in edge_to_ods[e] for g in in_indices ) \
                                == x[(e, 1, t)] ]

            ## Group flow constraints:
            for od in edge_to_ods[e]:
                for g in el_indices:
#                     print("(od, g, e, t):", od, g, e, t)
                    assert (od, g, e, t) not in od_g_e_t_list, "Each (od, g, e, t) should occur only once."
                    od_g_e_t_list += [(od, g, e, t)]
        
#                     if (od, g, e, t) == (0, 0, 0, 0):
#                     if (e, t) == (2, 0) or (2, 1) or (2, 3):
#                         constraints += [sum(y[(od, g, e, k, t)] for k in [0, 1, 2]) == demand_array[od, g]]
                    
#                 for g in in_indices:
# #                     print("(od, g, e, t):", od, g, e, t)
#                     assert (od, g, e, t) not in od_g_e_t_list, "Each (od, g, e, t) should occur only once."
#                     od_g_e_t_list += [(od, g, e, t)]
                    
#                     constraints += [sum(y[(od, g, e, k, t)] for k in [0, 1]) == demand_array[od, g]]

            constraints += [sum(y[(od, g, e, k, t)] for k in [0, 1, 2]) == demand_array[od, g] \
                            for od in edge_to_ods[e] for g in el_indices]
            constraints += [sum(y[(od, g, e, k, t)] for k in [0, 1]) == demand_array[od, g] \
                            for od in edge_to_ods[e] for g in in_indices]
    
    # Problem:
    prob = cp.Problem(objective, constraints)
    
    # Solve:
#     result = prob.solve(solver = cp.MOSEK, verbose = True)
    result = prob.solve(solver = cp.MOSEK)
    
#     for variable in prob.variables():
#         print("Variable %s" % variable.name())
    
    assert prob.status != "infeasible", "problem.status should not be infeasible."
    assert prob.status != "unbounded", "problem.status should not be unbounded."
    print()
    print("prob.status:", prob.status)

    # Extract Values:
    y_values = {}
    for e in range(num_edges):
        for od in edge_to_ods[e]:
            for t in range(T):
                for g in el_indices:
                    for k in [0, 1, 2]:
#                         print("(od, g, e, k, t):", od, g, e, k, t)
#                         print("y[(od, g, e, k, t)].value:", y[(od, g, e, k, t)].value)
                        y_values[(od, g, e, k, t)] = max(y[(od, g, e, k, t)].value[0], 0.0)
                for g in in_indices:
                    for k in [0, 1]:
#                         print("(od, g, e, k, t):", od, g, e, k, t)
#                         print("y[(od, g, e, k, t)].value:", y[(od, g, e, k, t)].value)
                        y_values[(od, g, e, k, t)] = max(y[(od, g, e, k, t)].value[0], 0.0)

    return y_values


## <font color='blue'>Implement Zeroth-order Gradient Descent:</font> 

In [20]:
## Functions defined above:

# def proj_tau_alpha(T, num_edges, tau, alpha, od_to_edges_list_full, tau_max = 1.0, alpha_max = 1.0):

# def solve_DBCP_direct(T, num_edges, num_gp_lanes, tau, alpha, od_to_edges_array, \
#                       demand_array, VoT_array, num_el, coeff_input):

#     def welfare_obj(T, num_edges, num_gp_lanes, lambda_E, lambda_R, lambda_I, tau, alpha \
#                 demand_array, VoT_array, num_el, od_to_edges_array, y, \
#                 coeff_input = np.array([0.0, 0.0, 0.0, 0.0, 1.0])):

In [21]:
# coeff_input
# demand_array.shape

In [22]:

T = 5

# num_iters = 1000
# num_iters = 100
num_iters = 200
# num_iters = 3

num_el = 2
num_groups = demand_array.shape[1]

assert VoT_array.shape[2] == T, "We should have VoT_array.shape[2] == T"
assert demand_array.shape == VoT_array[:, :, 0].shape, "demand_array and VoT_array[:, :, 0] should have the same shape."

num_ods = demand_array.shape[0]
group_indices = list(range(demand_array.shape[1]))
# num_edges = 6
num_edges = 7
num_gp_lanes = 3

# tau = np.array([0.2, 0.6, 0.5, 0.8, 0.4, 0.5])
# assert tau.shape[0] == num_edges, "tau must have a num_edges number of entries."

a = np.array([1E-3, 1E-3, 0.0, 0.0, 0.0])

error_bound = 1E-2
diffs_num_cols = 5

demand_edges_array = np.zeros(num_edges)

# demand_array_temp = np.ones(demand_array.shape)
# demand_array_temp = np.random.uniform(low=0.05, high=0.5, size=demand_array.shape)
demand_array_temp = demand_array


# TODO: Modify coeff_input:

## coeff_input: const, slope, x-coordinate of transition point

# coeff_input = np.array([19.4, 0.1256, 0.786*1650]).reshape((3, 1)) @ np.ones((1, num_edges))


coeff_input = np.zeros((3, num_edges))
for counter, city in enumerate(dict_latency_params.keys()):
    coeff_input[0, counter] = dict_latency_params[city]["Latency (at bend)"]
    coeff_input[1, counter] = dict_latency_params[city]["Slope (after bend)"]
    coeff_input[2, counter] = dict_latency_params[city]["Flow (at bend)"]


# def solve_DBCP_direct(T, num_edges, num_gp_lanes, tau, alpha, od_to_edges_array, \
#                       demand_array, VoT_array, num_el, coeff_input):
# Already defined: T, num_edges, num_gp_lanes, tau, alpha, od_to_edges_array, \
#                       demand_array, VoT_array

    
# print("od_to_edges_array[1, 0]:", int(od_to_edges_array[1, 0]))
# print("range(od_to_edges_array[1, 0], od_to_edges_array[1, 1] + 1):",\
#      range(int(od_to_edges_array[1, 0]), int(od_to_edges_array[1, 1]) + 1))

od_to_edges_list_full = [list(range(int(od_to_edges_array[od, 0]), int(od_to_edges_array[od, 1]) + 1)) \
                         for od in range(num_ods)]
edge_to_ods = [[od for od in range(od_to_edges_array.shape[0]) if e in od_to_edges_list_full[od]] \
               for e in range(num_edges)]
for e in range(num_edges):
    demand_edges_array[e] = sum([np.sum(demand_array_temp[od, :]) for od in range(num_ods) \
                                 if od in edge_to_ods[e]])

el_indices = list(range(num_el))
in_indices = list(range(num_el, num_groups))

tau_max_from_latency = np.zeros((num_edges, T))
VoT_max_el = np.zeros((num_edges, T))
for e in range(num_edges):
    for t in range(T):
        VoT_max_el[e, t] = max([VoT_array[od_index, group_index, t] for od_index in edge_to_ods[e] \
                                for group_index in el_indices])
        tau_max_from_latency[e, t] = VoT_max_el[e, t] * (latency_max(demand_edges_array[e], coeff_input[:, e]) - coeff_input[0, e]) \

tau_max_monetary_value = 15.0
# fraction_tau_max = tau_max_monetary_value / np.max(tau_max_from_latency)

tau_upper_limit = np.minimum(tau_max_from_latency, tau_max_monetary_value)

print("tau_upper_limit:", tau_upper_limit)

d = num_edges * T * 2

tau = np.zeros((num_edges, T, num_iters))
tau_perturbed = np.zeros((num_edges, T, num_iters))

alpha = np.zeros((num_edges, T, num_iters))
alpha_perturbed = np.zeros((num_edges, T, num_iters))

fraction_tau_max = 0.5

# tau[:, :, 0] = np.random.triangular(np.zeros((num_edges, T)), tau_upper_limit * fraction_tau_max, tau_upper_limit)
# print()
# print("tau[:, :, 0]:", tau[:, :, 0])
# print()
# print("np.sum(tau[:, :, 0]):", np.sum(tau[:, :, 0]))

od_to_edges_list_full = [list(range(int(od_to_edges_array[od, 0]), int(od_to_edges_array[od, 1]) + 1 )) \
                         for od in range(od_to_edges_array.shape[0])]


# # To disable when restarting from scratch
# tau[:, :, 0] = tau_next_init
# alpha[:, :, 0] = alpha_next_init
# eta[0] = eta_bar * (index_next_init+1)**(-1/2) * d**(-1)
# delta[0] = delta_bar * (index_next_init+1)**(-1/4) * d**(-1/2)

tau_upper_limit: [[0.31942364 0.31942364 0.31942364 0.31942364 0.31942364]
 [0.24983105 0.24983105 0.24983105 0.24983105 0.24983105]
 [0.99953381 0.99953381 0.99953381 0.99953381 0.99953381]
 [1.02805916 1.02805916 1.02805916 1.02805916 1.02805916]
 [1.6042613  1.6042613  1.6042613  1.6042613  1.6042613 ]
 [0.19224693 0.19224693 0.19224693 0.19224693 0.19224693]
 [0.21782897 0.21782897 0.21782897 0.21782897 0.21782897]]


In [23]:
time_1 = time.time()

welfare_list = []

# Set lambdas:

lambda_E, lambda_R, lambda_I = 1.0, 2.0, 5.0

## Initialize tau, alpha values:

filename_segment = str(int(lambda_E)) + '_' + str(int(lambda_R)) + '_' + str(int(lambda_I))

directory_inits = '../data/opt_tolls_subsidies_metrics/'
df_inits = pd.read_csv(directory_inits + 'inits___' + filename_segment + '.csv')

print("filename_segment:", filename_segment)

inits_tau_arr_as_object = df_inits.to_numpy()[:, 1:6]
inits_alpha_arr_as_object = df_inits.to_numpy()[:, -5:]

for e in range(num_edges):
    for t in range(T):
        tau[e, t, 0] = inits_tau_arr_as_object[e, t]
        alpha[e, t, 0] = inits_alpha_arr_as_object[e, t]

print()
print("tau[:, :, 0]:", tau[:, :, 0])
print("alpha[:, :, 0]:", alpha[:, :, 0])


## Rest of code, parallel to CBCP code:

delta = np.zeros(num_iters)
eta = np.zeros(num_iters)
eta_bar = 1E-5
delta_bar = 0.01

for i in range(num_iters-1):
    
    print()
    print("Iter:", i)
    
    eta[i] = eta_bar * (i+1)**(-1/2) * d**(-1)
    delta[i] = delta_bar * (i+1)**(-1/4) * d**(-1/2)
    w_i_unnormalized = np.random.randn(num_edges, T, 2)
    w_i = w_i_unnormalized / np.linalg.norm(w_i_unnormalized)
#     print("w_i:", w_i)
    tau_perturbed[:, :, i] = tau[:, :, i] + delta[i] * w_i[:, :, 0]
    alpha_perturbed[:, :, i] = alpha[:, :, i] + delta[i] * w_i[:, :, 1]
    
#     if tau_perturbed[:, :, i] < alpha_perturbed[:, :, i] or tau_perturbed[:, :, i] < 0 alpha_perturbed[:, :, i] < 0:

    print("i:\n", i)
    print("tau[:, :, i], before projection:\n", tau[:, :, i])
    print("alpha[:, :, i], before projection:\n", alpha[:, :, i])
    print("tau_perturbed[:, :, i], before projection:\n", tau_perturbed[:, :, i])
    print("alpha_perturbed[:, :, i], before projection:\n", alpha_perturbed[:, :, i])

    tau_perturbed[:, :, i], alpha_perturbed[:, :, i] \
        = proj_tau_alpha(T, num_edges, tau_perturbed[:, :, i], alpha_perturbed[:, :, i], \
                         od_to_edges_list_full, tau_max = tau_upper_limit)
    
    print("tau[:, :, i], after projection:\n", tau[:, :, i])
    print("alpha[:, :, i], after projection:\n", alpha[:, :, i])
    print("tau_perturbed[:, :, i], after projection:\n", tau_perturbed[:, :, i])
    print("alpha_perturbed[:, :, i], after projection:\n", alpha_perturbed[:, :, i])

    y_values = solve_DBCP_direct(T, num_edges, num_gp_lanes, \
                                 tau[:, :, i], alpha[:, :, i], od_to_edges_array, \
                                 demand_array_temp, VoT_array, num_el, coeff_input)

    y_perturbed_values = solve_DBCP_direct(T, num_edges, num_gp_lanes, \
                                           tau_perturbed[:, :, i], alpha_perturbed[:, :, i], od_to_edges_array, \
                                           demand_array_temp, VoT_array, num_el, coeff_input)
    
#     def welfare_obj(T, num_edges, num_gp_lanes, lambda_E, lambda_R, lambda_I, tau, alpha \
#                 demand_array, VoT_array, num_el, od_to_edges_array, y, \
#                 coeff_input):

    welfare, obj_E, obj_R, obj_I = welfare_obj(T, num_edges, num_gp_lanes, lambda_E, lambda_R, lambda_I, \
                                               tau[:, :, i], alpha[:, :, i], demand_array_temp, VoT_array, num_el, od_to_edges_array, \
                                               y = y_values, coeff_input = coeff_input)
    
    welfare_perturbed, obj_E_perturbed, obj_R_perturbed, obj_I_perturbed \
        = welfare_obj(T, num_edges, num_gp_lanes, lambda_E, lambda_R, lambda_I, \
                      tau_perturbed[:, :, i], alpha_perturbed[:, :, i], demand_array_temp, VoT_array, num_el, od_to_edges_array, \
                      y = y_perturbed_values, coeff_input = coeff_input)
    
    print("welfare:", welfare)
    print("obj_E:", obj_E)
    print("obj_R:", obj_R)
    print("obj_I:", obj_I)
    print()
    print("welfare_perturbed:", welfare_perturbed)
    print("obj_E_perturbed:", obj_E_perturbed)
    print("obj_R_perturbed:", obj_R_perturbed)
    print("obj_I_perturbed:", obj_I_perturbed)
    print()
    
    welfare_list.append(welfare)
    
    tau[:, :, i+1] = tau[:, :, i] - eta[i] * (d/delta[i]) * w_i[:, :, 0] \
                        * (welfare_perturbed - welfare)
    
    alpha[:, :, i+1] = alpha[:, :, i] - eta[i] * (d/delta[i]) * w_i[:, :, 1] * (welfare_perturbed - welfare)
    
    print("tau[:, :, i+1], before projection:\n", tau[:, :, i+1])
    print("alpha[:, :, i+1], before projection:\n", alpha[:, :, i+1])
    
    tau[:, :, i+1], alpha[:, :, i+1] = proj_tau_alpha(T, num_edges, tau[:, :, i+1], alpha[:, :, i+1], od_to_edges_list_full, \
                                                      tau_max = tau_upper_limit)
    
    print("tau[:, :, i+1], after projection:\n", tau[:, :, i+1])
    print("alpha[:, :, i+1], after projection:\n", alpha[:, :, i+1])
    
    if i >= diffs_num_cols + 2:
        tau_diffs = np.max(np.absolute(tau[:, :, i - diffs_num_cols : i-1] - tau[:, :, i - diffs_num_cols+1 : i]), axis = 2)
        alpha_diffs = alpha[:, :, i-diffs_num_cols : i-1] - alpha[:, :, i-diffs_num_cols+1 : i]
        
#         print("tau[:, :, 0:10]:", tau[:, :, 0:10])
#         print("alpha[:, :, 0:10]:", alpha[:, :, 0:10])
        print("tau_diffs:\n", tau_diffs)
        print("alpha_diffs:\n", alpha_diffs)
        print()
        
        if max(np.max(np.absolute(tau_diffs)), np.max(np.absolute(alpha_diffs))) < error_bound:
            print("Within error bound.")
            break

    time_2 = time.time()

    print()
    print("Iteration count:", i)
    print("Time:", time_2 - time_1)
    print()



filename_segment: 1_2_5

tau[:, :, 0]: [[0.     0.3194 0.066  0.3194 0.3194]
 [0.2498 0.2498 0.2498 0.     0.103 ]
 [0.0911 0.9995 0.9995 0.     0.    ]
 [0.     0.     0.     0.     0.5557]
 [0.7473 0.     0.6926 1.6043 0.253 ]
 [0.     0.     0.     0.1922 0.    ]
 [0.2178 0.2178 0.2178 0.0376 0.2178]]
alpha[:, :, 0]: [[0.         0.95167855 0.98244702 0.95067844 0.95027234]
 [0.95203821 0.95165467 0.95086844 0.         0.94840715]
 [0.94250443 0.94321787 0.94189516 0.         0.        ]
 [0.         0.         0.         0.         0.94358402]
 [0.94250443 0.         0.93951487 0.9415683  0.94358402]
 [0.         0.         0.         0.94273842 0.        ]
 [0.94250443 0.94196803 0.94194071 0.9405564  0.94358402]]

Iter: 0
i:
 0
tau[:, :, i], before projection:
 [[0.     0.3194 0.066  0.3194 0.3194]
 [0.2498 0.2498 0.2498 0.     0.103 ]
 [0.0911 0.9995 0.9995 0.     0.    ]
 [0.     0.     0.     0.     0.5557]
 [0.7473 0.     0.6926 1.6043 0.253 ]
 [0.     0.     0.     0.1922 0.


prob.status: optimal

prob.status: optimal

in_indices: [2, 3, 4]
el_indices: [0, 1]


in_indices: [2, 3, 4]
el_indices: [0, 1]

welfare: 3873911.9585130345
obj_E: 6007.329847057086
obj_R: 6080.433157247688
obj_I: 776013.0989960945

welfare_perturbed: 3873911.9585130345
obj_E_perturbed: 6007.329847057086
obj_R_perturbed: 6080.433157247688
obj_I_perturbed: 776013.0989960945

tau[:, :, i+1], before projection:
 [[0.    0.    0.    0.319 0.   ]
 [0.25  0.25  0.    0.25  0.   ]
 [1.    1.    1.    0.    0.   ]
 [1.028 0.    1.028 0.    1.028]
 [1.604 0.    1.604 1.604 0.   ]
 [0.192 0.192 0.    0.    0.   ]
 [0.    0.218 0.    0.218 0.218]]
alpha[:, :, i+1], before projection:
 [[1. 1. 0. 1. 1.]
 [1. 1. 1. 1. 1.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 1.]
 [0. 0. 1. 0. 0.]
 [1. 1. 1. 1. 0.]
 [1. 0. 0. 0. 1.]]

tau_feas.value: [[-0.    -0.    -0.     0.319 -0.   ]
 [ 0.25   0.25  -0.     0.25  -0.   ]
 [ 1.     1.     1.    -0.    -0.   ]
 [ 1.028 -0.     1.028 -0.     1.028]
 [ 1.604 -0.     1.6


prob.status: optimal

prob.status: optimal

in_indices: [2, 3, 4]
el_indices: [0, 1]


in_indices: [2, 3, 4]
el_indices: [0, 1]

welfare: 3873911.9585130345
obj_E: 6007.329847057086
obj_R: 6080.433157247688
obj_I: 776013.0989960945

welfare_perturbed: 3873911.9585130345
obj_E_perturbed: 6007.329847057086
obj_R_perturbed: 6080.433157247688
obj_I_perturbed: 776013.0989960945

tau[:, :, i+1], before projection:
 [[ 0.     0.     0.     0.319 -0.   ]
 [ 0.25   0.25  -0.     0.25   0.   ]
 [ 1.     1.     1.    -0.    -0.   ]
 [ 1.028  0.     1.028  0.     1.028]
 [ 1.604 -0.     1.604  1.604 -0.   ]
 [ 0.192  0.192 -0.    -0.     0.   ]
 [ 0.     0.218 -0.     0.218  0.218]]
alpha[:, :, i+1], before projection:
 [[1. 1. 0. 1. 1.]
 [1. 1. 1. 1. 1.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 1.]
 [0. 0. 1. 0. 0.]
 [1. 1. 1. 1. 0.]
 [1. 0. 0. 0. 1.]]

tau_feas.value: [[-0.    -0.    -0.     0.319 -0.   ]
 [ 0.25   0.25  -0.     0.25  -0.   ]
 [ 1.     1.     1.    -0.    -0.   ]
 [ 1.028 -0.     1.028 


prob.status: optimal

prob.status: optimal

in_indices: [2, 3, 4]
el_indices: [0, 1]


in_indices: [2, 3, 4]
el_indices: [0, 1]

welfare: 3873911.9585130345
obj_E: 6007.329847057086
obj_R: 6080.433157247688
obj_I: 776013.0989960945

welfare_perturbed: 3873911.9585130345
obj_E_perturbed: 6007.329847057086
obj_R_perturbed: 6080.433157247688
obj_I_perturbed: 776013.0989960945

tau[:, :, i+1], before projection:
 [[ 0.    -0.    -0.     0.319 -0.   ]
 [ 0.25   0.25  -0.     0.25   0.   ]
 [ 1.     1.     1.    -0.     0.   ]
 [ 1.028  0.     1.028 -0.     1.028]
 [ 1.604  0.     1.604  1.604 -0.   ]
 [ 0.192  0.192  0.    -0.     0.   ]
 [ 0.     0.218  0.     0.218  0.218]]
alpha[:, :, i+1], before projection:
 [[1. 1. 0. 1. 1.]
 [1. 1. 1. 1. 1.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 1.]
 [0. 0. 1. 0. 0.]
 [1. 1. 1. 1. 0.]
 [1. 0. 0. 0. 1.]]

tau_feas.value: [[-0.    -0.    -0.     0.319 -0.   ]
 [ 0.25   0.25  -0.     0.25  -0.   ]
 [ 1.     1.     1.    -0.    -0.   ]
 [ 1.028 -0.     1.028 


prob.status: optimal

prob.status: optimal

in_indices: [2, 3, 4]
el_indices: [0, 1]


in_indices: [2, 3, 4]
el_indices: [0, 1]

welfare: 3873911.9585130345
obj_E: 6007.329847057086
obj_R: 6080.433157247688
obj_I: 776013.0989960945

welfare_perturbed: 3873911.9585130345
obj_E_perturbed: 6007.329847057086
obj_R_perturbed: 6080.433157247688
obj_I_perturbed: 776013.0989960945

tau[:, :, i+1], before projection:
 [[ 0.     0.     0.     0.319  0.   ]
 [ 0.25   0.25   0.     0.25   0.   ]
 [ 1.     1.     1.    -0.     0.   ]
 [ 1.028 -0.     1.028 -0.     1.028]
 [ 1.604  0.     1.604  1.604 -0.   ]
 [ 0.192  0.192  0.     0.    -0.   ]
 [ 0.     0.218 -0.     0.218  0.218]]
alpha[:, :, i+1], before projection:
 [[1. 1. 0. 1. 1.]
 [1. 1. 1. 1. 1.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 1.]
 [0. 0. 1. 0. 0.]
 [1. 1. 1. 1. 0.]
 [1. 0. 0. 0. 1.]]

tau_feas.value: [[-0.    -0.    -0.     0.319 -0.   ]
 [ 0.25   0.25  -0.     0.25  -0.   ]
 [ 1.     1.     1.    -0.    -0.   ]
 [ 1.028 -0.     1.028 

In [24]:
# dict_latency_params["Belmont"]

In [25]:
min_welfare = min(welfare_list)
argmin_welfare_list = welfare_list.index(min(welfare_list))
argmin_tau = tau[:, :, argmin_welfare_list]
argmin_alpha = alpha[:, :, argmin_welfare_list]

# print("argmin_alpha:", argmin_alpha)
# print("alpha[:, :, argmin_welfare_list]:", alpha[:, :, argmin_welfare_list])
# print("min_welfare:", min_welfare)
# print()
print(welfare_list)

print()
print(min_welfare)
print()
print(argmin_welfare_list)
print()
# print("len(welfare_list)", len(welfare_list))
print("argmin_alpha:", argmin_alpha)
print()
print("argmin_tau:", argmin_tau)
print()


# welfare_list[argmin_welfare_list]
# alpha[:, :, argmin_welfare_list]

[3671825.6448050328, 3873911.9585130345, 3873911.9585130345, 3873911.9585130345, 3873911.9585130345, 3873911.9585130345, 3873911.9585130345, 3873911.9585130345]

3671825.6448050328

0

argmin_alpha: [[0.         0.95167855 0.98244702 0.95067844 0.95027234]
 [0.95203821 0.95165467 0.95086844 0.         0.94840715]
 [0.94250443 0.94321787 0.94189516 0.         0.        ]
 [0.         0.         0.         0.         0.94358402]
 [0.94250443 0.         0.93951487 0.9415683  0.94358402]
 [0.         0.         0.         0.94273842 0.        ]
 [0.94250443 0.94196803 0.94194071 0.9405564  0.94358402]]

argmin_tau: [[0.     0.3194 0.066  0.3194 0.3194]
 [0.2498 0.2498 0.2498 0.     0.103 ]
 [0.0911 0.9995 0.9995 0.     0.    ]
 [0.     0.     0.     0.     0.5557]
 [0.7473 0.     0.6926 1.6043 0.253 ]
 [0.     0.     0.     0.1922 0.    ]
 [0.2178 0.2178 0.2178 0.0376 0.2178]]



In [26]:
# index_next_init = argmin_welfare_list
# alpha_next_init = alpha[:, :, index_next_init]
# tau_next_init = tau[:, :, index_next_init]

# print("index_next_init:", index_next_init)
# print("alp[ha_next_init:", alpha_next_init)
# print("tau_next_init:", tau_next_init)


In [27]:
# lambda combinations to try to simulate:
# (1, 1, 1)
# (5, 5, 1)
# (10, 10, 1)
# (5, 10, 1)
# (5, 15, 1)
# (10, 15, 1)
# (10, 20, 1)
# (20, 20, 1)
# (20, 30, 1)
# (20, 40, 1)

In [28]:

# argmin_tau = np.array([[0.7091, 1.8252, 0.3238, 0.1215, 0.1079],
#                        [1.8739, 0.7429, 0.0, 0.9112, 1.8739],
#                        [0.0, 3.0448, 0.0, 0.0, 3.9494],
#                        [2.9504, 2.377, 3.4944, 0.0, 0.0],
#                        [7.2392, 8.4126, 0.0, 6.819, 0.6135],
#                        [0.2836, 0.8496, 0.963, 0.9527, 0.5689],
#                        [0.1346, 0.0, 0.0, 0.0712, 0.5024]]) / 100

# # argmin_alpha = 0.1664
# argmin_alpha = 0.0

# # VoT_array[:, :, 0]

In [29]:
argmin_y = solve_DBCP_direct(T, num_edges, num_gp_lanes, \
                                    argmin_tau, argmin_alpha, od_to_edges_array, \
                                    demand_array_temp, VoT_array, num_el, coeff_input)

print("argmin_tau:", argmin_tau)
print("argmin_alpha:", argmin_alpha)
print("min_welfare:", min_welfare)
print()
print("argmin_y:\n")

argmin_y



prob.status: optimal
argmin_tau: [[0.     0.3194 0.066  0.3194 0.3194]
 [0.2498 0.2498 0.2498 0.     0.103 ]
 [0.0911 0.9995 0.9995 0.     0.    ]
 [0.     0.     0.     0.     0.5557]
 [0.7473 0.     0.6926 1.6043 0.253 ]
 [0.     0.     0.     0.1922 0.    ]
 [0.2178 0.2178 0.2178 0.0376 0.2178]]
argmin_alpha: [[0.         0.95167855 0.98244702 0.95067844 0.95027234]
 [0.95203821 0.95165467 0.95086844 0.         0.94840715]
 [0.94250443 0.94321787 0.94189516 0.         0.        ]
 [0.         0.         0.         0.         0.94358402]
 [0.94250443 0.         0.93951487 0.9415683  0.94358402]
 [0.         0.         0.         0.94273842 0.        ]
 [0.94250443 0.94196803 0.94194071 0.9405564  0.94358402]]
min_welfare: 3671825.6448050328

argmin_y:



{(0, 0, 0, 0, 0): -0.0,
 (0, 0, 0, 1, 0): -0.0,
 (0, 0, 0, 2, 0): 47.77809012103125,
 (0, 1, 0, 0, 0): -0.0,
 (0, 1, 0, 1, 0): -0.0,
 (0, 1, 0, 2, 0): 27.564282762133406,
 (0, 2, 0, 0, 0): -0.0,
 (0, 2, 0, 1, 0): 115.15744798402402,
 (0, 3, 0, 0, 0): -0.0,
 (0, 3, 0, 1, 0): 134.75871572598555,
 (0, 4, 0, 0, 0): -0.0,
 (0, 4, 0, 1, 0): 287.28108034312373,
 (0, 0, 0, 0, 1): 0.00016836073977495803,
 (0, 0, 0, 1, 1): -0.0,
 (0, 0, 0, 2, 1): 47.77792176029147,
 (0, 1, 0, 0, 1): -0.0,
 (0, 1, 0, 1, 1): -0.0,
 (0, 1, 0, 2, 1): 27.564282762133406,
 (0, 2, 0, 0, 1): 6.749353701999252e-05,
 (0, 2, 0, 1, 1): 115.157380490487,
 (0, 3, 0, 0, 1): 0.00012237074352928725,
 (0, 3, 0, 1, 1): 134.75859335524203,
 (0, 4, 0, 0, 1): 0.001534414342324452,
 (0, 4, 0, 1, 1): 287.27954592878143,
 (0, 0, 0, 0, 2): -0.0,
 (0, 0, 0, 1, 2): -0.0,
 (0, 0, 0, 2, 2): 47.77809012103125,
 (0, 1, 0, 0, 2): 27.564282762133416,
 (0, 1, 0, 1, 2): -0.0,
 (0, 1, 0, 2, 2): 0.0,
 (0, 2, 0, 0, 2): 0.00032290454618646275,
 (0, 2,

In [30]:
# demand_array_temp

# tau_upper_limit
# T
# num_edges
# num_gp_lanes
# alpha_perturbed[:, :, i]
# coeff_input
# od_to_edges_array
# demand_array_temp
# VoT_array
# num_el
# coeff_input

# np.minimum(np.array([1, 2]), 0.5)

# tau_upper_limit
# argmin_welfare_list

# Compute Cost Metrics

In [42]:
num_groups = demand_array.shape[1]

el_indices = list(range(num_el))
in_indices = list(range(num_el, num_groups))

edge_to_od_dict = {}
for e in range(num_edges):
    edge_to_od_dict[e] = [k for k in list(range(int(od_to_edges_array.shape[0]) )) \
                           if od_to_edges_array[k, 0] <= e <= od_to_edges_array[k, 1]]

argmin_x = {}
for e in range(num_edges):
    for t in range(T):
        argmin_x[(e, 0, t)] = 0
        argmin_x[(e, 0, t)] += sum(argmin_y[(od, g, e, 0, t)] + argmin_y[(od, g, e, 1, t)] \
                                   for od in edge_to_od_dict[e] for g in el_indices)
        argmin_x[(e, 0, t)] += sum(argmin_y[(od, g, e, 0, t)] \
                                   for od in edge_to_od_dict[e] for g in in_indices)
        
        argmin_x[(e, 1, t)] = 0
        argmin_x[(e, 1, t)] += sum(argmin_y[(od, g, e, 2, t)] \
                                   for od in edge_to_od_dict[e] for g in el_indices)
        argmin_x[(e, 1, t)] += sum(argmin_y[(od, g, e, 1, t)] \
                                   for od in edge_to_od_dict[e] for g in in_indices)

argmin_y_in_el_total = {}      
for e in range(num_edges):
    for t in range(T):
        argmin_y_in_el_total[(e, 0, t, 'el')] = sum(argmin_y[(od, g, e, 0, t)] + argmin_y[(od, g, e, 1, t)] \
                                                    for od in edge_to_od_dict[e] for g in el_indices)
        
        argmin_y_in_el_total[(e, 1, t, 'el')] = sum(argmin_y[(od, g, e, 2, t)] \
                                                    for od in edge_to_od_dict[e] for g in el_indices)
        
        argmin_y_in_el_total[(e, 0, t, 'in')] = sum(argmin_y[(od, g, e, 0, t)] \
                                                    for od in edge_to_od_dict[e] for g in in_indices)
        
        argmin_y_in_el_total[(e, 1, t, 'in')] = sum(argmin_y[(od, g, e, 1, t)] \
                                                    for od in edge_to_od_dict[e] for g in in_indices)


# y[(od, g, e, 0, t)] + y[(od, g, e, 1, t)] for od in edge_to_od_dict[e] for g in el_indice

# argmin_x

argmin_y_in_el_total

{(0, 0, 0, 'el'): 0.0,
 (0, 1, 0, 'el'): 596.3167467964665,
 (0, 0, 0, 'in'): 1002.0023693338987,
 (0, 1, 0, 'in'): 3249.784516361232,
 (0, 0, 1, 'el'): 0.00016836073977495803,
 (0, 1, 1, 'el'): 596.3165784357268,
 (0, 0, 1, 'in'): 528.7609534189351,
 (0, 1, 1, 'in'): 3723.0259322761963,
 (0, 0, 2, 'el'): 266.8652067230498,
 (0, 1, 2, 'el'): 329.45154007341677,
 (0, 0, 2, 'in'): 693.4805189487902,
 (0, 1, 2, 'in'): 3558.3063667463402,
 (0, 0, 3, 'el'): 218.16482833438013,
 (0, 1, 3, 'el'): 378.1519184620865,
 (0, 0, 3, 'in'): 274.7167283397624,
 (0, 1, 3, 'in'): 3977.070157355369,
 (0, 0, 4, 'el'): 4.411734964330892,
 (0, 1, 4, 'el'): 591.9050118321356,
 (0, 0, 4, 'in'): 473.9030823127544,
 (0, 1, 4, 'in'): 3777.883803382377,
 (1, 0, 0, 'el'): 83.49029377084995,
 (1, 1, 0, 'el'): 575.4681043972175,
 (1, 0, 0, 'in'): 285.2675845020103,
 (1, 1, 0, 'in'): 4230.004864550294,
 (1, 0, 1, 'el'): 0.8547210921759629,
 (1, 1, 1, 'el'): 658.1036770758915,
 (1, 0, 1, 'in'): 353.5024291820891,
 (1,

In [43]:
# coeff_input

print("argmin_tau[5, 3]:", argmin_tau[5, 3])
print("argmin_x[(5, 0, 3)]:", argmin_x[(5, 0, 3)])
print("argmin_x[(5, 1, 3)]:", argmin_x[(5, 1, 3)])


argmin_tau[5, 3]: 0.1922
argmin_x[(5, 0, 3)]: 872.7885945025557
argmin_x[(5, 1, 3)]: 5772.282128753137


In [44]:
travel_times = {}

## coeff_input: const, slope, x-coordinate of transition point
# coeff_input = np.array([19.4, 0.1256, 0.786*1650]).reshape((3, 1)) @ np.ones((1, num_edges))

for e in range(num_edges):
    for t in range(T):
        travel_times[(e, 0, t)] = coeff_input[0, e] + coeff_input[1, e] * max(argmin_x[(e, 0, t)] - coeff_input[2, e], 0.0)
        travel_times[(e, 1, t)] = coeff_input[0, e] + coeff_input[1, e] * max(argmin_x[(e, 1, t)]/num_gp_lanes - coeff_input[2, e], 0.0)

edge_demand = {}
avg_travel_time = {}
percent_on_express = {}
obj_E = np.zeros(num_edges)
obj_I = np.zeros(num_edges)
obj_R = np.zeros(num_edges)
obj = np.zeros(num_edges)

for e in range(num_edges):
    avg_travel_time[e, 'el'] = 0.0
    avg_travel_time[e, 'in'] = 0.0
    avg_travel_time[e, 'ex'] = 0.0
    avg_travel_time[e, 'gp'] = 0.0
    
    percent_on_express[e, 'el'] = 0.0
    percent_on_express[e, 'in'] = 0.0
    percent_on_express[e, 'all'] = 0.0
    
    obj_E[e] = 0.0
    obj_I[e] = 0.0
    obj_R[e] = 0.0
    obj[e] = 0.0
    
    for t in range(T):
        edge_demand[e, t, 'el'] = sum(argmin_y_in_el_total[(e, k, t, 'el')] for k in range(2))
        edge_demand[e, t, 'in'] = sum(argmin_y_in_el_total[(e, k, t, 'in')] for k in range(2))
        
    percent_on_express[e, 'el'] += sum(argmin_y_in_el_total[(e, 0, t, 'el')] for t in range(T)) \
                                    / sum(edge_demand[e, t, 'el'] for t in range(T))
    percent_on_express[e, 'in'] += sum(argmin_y_in_el_total[(e, 0, t, 'in')] for t in range(T)) \
                                    / sum(edge_demand[e, t, 'in'] for t in range(T))
    percent_on_express[e, 'all'] += sum(argmin_y_in_el_total[(e, 0, t, 'el')] + argmin_y_in_el_total[(e, 0, t, 'in')] for t in range(T)) \
                                    / sum(edge_demand[e, t, 'el'] + edge_demand[e, t, 'in'] for t in range(T))
    
    avg_travel_time[e, 'el'] += sum(argmin_y_in_el_total[(e, k, t, 'el')] * travel_times[(e, k, t)] for k in range(2) for t in range(T)) \
                                    / sum(edge_demand[e, t, 'el'] for t in range(T))    
    avg_travel_time[e, 'in'] += sum(argmin_y_in_el_total[(e, k, t, 'in')] * travel_times[(e, k, t)] for k in range(2) for t in range(T)) \
                                    / sum(edge_demand[e, t, 'in'] for t in range(T))    
#     avg_travel_time[e, 'ex'] += sum( (argmin_y_in_el_total[(e, 0, t, 'el')] + argmin_y_in_el_total[(e, 0, t, 'in')]) * travel_times[(e, 0, t)] for t in range(T)) \
#                                     / sum( argmin_y_in_el_total[(e, 0, t, 'el')] + argmin_y_in_el_total[(e, 0, t, 'in')] for t in range(T))
#     avg_travel_time[e, 'gp'] += sum( (argmin_y_in_el_total[(e, 1, t, 'el')] + argmin_y_in_el_total[(e, 1, t, 'in')]) * travel_times[(e, 0, t)] for t in range(T)) \
#                                     / sum( argmin_y_in_el_total[(e, 1, t, 'el')] + argmin_y_in_el_total[(e, 1, t, 'in')] for t in range(T))
    avg_travel_time[e, 'ex'] += sum(travel_times[(e, 0, t)] for t in range(T)) / T 
    avg_travel_time[e, 'gp'] += sum(travel_times[(e, 1, t)] for t in range(T)) / T 
    
    obj_E[e] = sum( argmin_y[(od, g, e, 0, t)] * (VoT_array[od, g, t] * travel_times[e, 0, t] + (1 - argmin_alpha[e, t]) * argmin_tau[e, t]) \
                        for od in edge_to_od_dict[e] for g in el_indices for t in range(T) ) \
                    + sum( argmin_y[(od, g, e, 1, t)] * (VoT_array[od, g, t] * travel_times[e, 0, t] + argmin_tau[e, t]) \
                        for od in edge_to_od_dict[e] for g in el_indices for t in range(T) ) \
                    + sum( argmin_y[(od, g, e, 2, t)] * VoT_array[od, g, t] * travel_times[e, 1, t] \
                          for od in edge_to_od_dict[e] for g in el_indices for t in range(T) ) 

    obj_I[e] = sum( argmin_y[(od, g, e, 0, t)] * (VoT_array[od, g, t] * travel_times[e, 0, t] + argmin_tau[e, t]) \
                        for od in edge_to_od_dict[e] for g in in_indices for t in range(T) ) \
                    + sum( argmin_y[(od, g, e, 1, t)] * VoT_array[od, g, t] * travel_times[e, 1, t] \
                        for od in edge_to_od_dict[e] for g in in_indices for t in range(T) )

    obj_R[e] = sum( argmin_y[(od, g, e, 0, t)] * argmin_tau[e, t] \
                        for od in edge_to_od_dict[e] for g in in_indices for t in range(T) ) \
                    + sum( argmin_y[(od, g, e, 1, t)] * argmin_tau[e, t] \
                        for od in edge_to_od_dict[e] for g in el_indices for t in range(T) ) \
                    + sum( argmin_y[(od, g, e, 0, t)] * (1 - argmin_alpha[e, t]) * argmin_tau[e, t] \
                        for od in edge_to_od_dict[e] for g in el_indices for t in range(T) )

    obj[e] = lambda_E * obj_E[e] - lambda_R * obj_R[e] + lambda_I * obj_I[e]


# welfare_obj(T, num_edges, num_gp_lanes, lambda_E, lambda_R, lambda_I, argmin_tau, argmin_alpha, \
#                 demand_array, VoT_array, num_el, od_to_edges_array, y, \
#                 coeff_input)

# avg_travel_time
# percent_on_express
# obj_R



In [45]:
# e = 0
# od = 0
# g = 0
# t = 0

# # sum( argmin_y[(od, g, e, 0, t)] * VoT_array[od, g, t] * travel_times[e, 0, t] \
# #                         for od in edge_to_od_dict[e] for g in el_indices for t in range(T) ) 
# sum( argmin_y[(od, g, e, 1, t)] * (VoT_array[od, g, t] * travel_times[e, 0, t] + tau[e, t]) \
#     for od in edge_to_od_dict[e] for g in el_indices for t in range(T) ) 
# #                     + sum( argmin_y[(od, g, e, 2, t)] * VoT_array[od, g, t] * travel_times[e, 1, t] \
# #                           for od in edge_to_od_dict[e] for g in el_indices for t in range(T) ) ) 

# # argmin_y[(od, g, e, 1, t)]



# # sum(edge_demand[e, t, 'el'] for t in range(T) )

In [46]:
# travel_times
argmin_x

{(0, 0, 0): 1002.0023693338987,
 (0, 1, 0): 3846.1012631576987,
 (0, 0, 1): 528.7611217796748,
 (0, 1, 1): 4319.342510711923,
 (0, 0, 2): 960.34572567184,
 (0, 1, 2): 3887.757906819757,
 (0, 0, 3): 492.88155667414253,
 (0, 1, 3): 4355.222075817455,
 (0, 0, 4): 478.3148172770853,
 (0, 1, 4): 4369.788815214513,
 (1, 0, 0): 368.75787827286024,
 (1, 1, 0): 4805.472968947512,
 (1, 0, 1): 354.35715027426505,
 (1, 1, 1): 4819.873696946108,
 (1, 0, 2): 324.8338748260013,
 (1, 1, 2): 4849.396972394371,
 (1, 0, 3): 1118.3327644677559,
 (1, 1, 3): 4055.8980827526157,
 (1, 0, 4): 1038.466175462539,
 (1, 1, 4): 4135.764671757833,
 (2, 0, 0): 1180.1914689079401,
 (2, 1, 0): 5613.5264612768615,
 (2, 0, 1): 905.951959383437,
 (2, 1, 1): 5887.765970801365,
 (2, 0, 2): 898.9111847179529,
 (2, 1, 2): 5894.806745466849,
 (2, 0, 3): 1208.0817123413226,
 (2, 1, 3): 5585.636217843477,
 (2, 0, 4): 1208.0817123413226,
 (2, 1, 4): 5585.636217843477,
 (3, 0, 0): 1372.1321044765705,
 (3, 1, 0): 4675.50619462393,


In [47]:
# print("argmin_x[(3, 0, 0)]:", argmin_x[(3, 0, 0)])
# print("argmin_x[(3, 1, 0)]:", argmin_x[(3, 1, 0)])

In [48]:
arr = np.array([[1, 2, 3, 4, 5, 6, 7, 8, 9, 10], \
                [-10, -8, -7, -5, -6, -9, -4, -3, -1, -2]])
print(arr)

arr[:, 3:8]

[[  1   2   3   4   5   6   7   8   9  10]
 [-10  -8  -7  -5  -6  -9  -4  -3  -1  -2]]


array([[ 4,  5,  6,  7,  8],
       [-5, -6, -9, -4, -3]])

In [49]:
opt_data_array = np.zeros((num_edges, 21))

# argmin_tau
opt_data_array[:, 0:5] = argmin_tau

# argmin_tau_avg
opt_data_array[:, 5] = np.mean(argmin_tau, axis=1)

# argmin_alpha
opt_data_array[:, 6:11] = argmin_alpha

# argmin_alpha_avg
opt_data_array[:, 11] = np.mean(argmin_alpha, axis=1)

# percent_on_express (overall)
# percent_on_express (eligible)
# percent_on_express (ineligible)
opt_data_array[:, 12] = np.array([percent_on_express[e, 'all'] for e in range(num_edges)]) * 100
opt_data_array[:, 13] = np.array([percent_on_express[e, 'el'] for e in range(num_edges)]) * 100
opt_data_array[:, 14] = np.array([percent_on_express[e, 'in'] for e in range(num_edges)]) * 100

# avg_travel_time (express lane)
# avg_travel_time (general purpose lane)
opt_data_array[:, 15] = np.array([avg_travel_time[e, 'ex'] for e in range(num_edges)])
opt_data_array[:, 16] = np.array([avg_travel_time[e, 'gp'] for e in range(num_edges)])

# obj_E = {}
# obj_I = {}
# obj_R = {}
# obj
opt_data_array[:, 17] = np.array([obj_E[e] for e in range(num_edges)]) 
opt_data_array[:, 18] = np.array([obj_I[e] for e in range(num_edges)]) 
opt_data_array[:, 19] = np.array([obj_R[e] for e in range(num_edges)]) 
opt_data_array[:, 20] = np.array([obj[e] for e in range(num_edges)]) 


opt_data_array = np.round(opt_data_array, decimals=2)

In [50]:
column_names = []
column_names += ["tau (t=" + str(t+1) + ")" for t in range(T) ]
column_names += ["tau (time-averaged)"]
column_names += ["alpha (t=" + str(t+1) + ")" for t in range(T) ]
column_names += ["alpha (time-averaged)"]
column_names += ["% overall users using express lanes", \
                 "% eligible users using express lanes", \
                 "% ineligible users using express lanes", \
                 "Average travel time (express lanes)", \
                 "Average travel time (general purpose lanes)", \
                 "Total travel cost (eligible users)", \
                 "Total travel cost (ineligible users)", \
                 "Total toll revenue", \
                 "Total societal cost"]

row_names = ["e=" + str(k+1) for k in range(num_edges) ]

df_opt_save = pd.DataFrame(opt_data_array, index=row_names, columns=column_names)

df_opt_save



,tau (t=1),tau (t=2),tau (t=3),tau (t=4),tau (t=5),tau (time-averaged),alpha (t=1),alpha (t=2),alpha (t=3),alpha (t=4),...,alpha (time-averaged),% overall users using express lanes,% eligible users using express lanes,% ineligible users using express lanes,Average travel time (express lanes),Average travel time (general purpose lanes),Total travel cost (eligible users),Total travel cost (ineligible users),Total toll revenue,Total societal cost
e=1,0.00,0.32,0.07,0.32,0.32,0.20,0.00,0.95,0.98,0.95,...,0.77,14.28,16.42,13.98,1.36,1.74,285.85,42549.37,457.58,212117.56
e=2,0.25,0.25,0.25,0.00,0.10,0.17,0.95,0.95,0.95,0.00,...,0.76,12.39,5.16,13.44,2.23,2.51,502.61,61446.25,340.17,307053.54
e=3,0.09,1.00,1.00,0.00,0.00,0.42,0.94,0.94,0.94,0.00,...,0.57,15.90,0.43,18.10,5.20,6.49,1747.96,201582.07,1894.54,1005869.23
e=4,0.00,0.00,0.00,0.00,0.56,0.11,0.00,0.00,0.00,0.00,...,0.19,21.13,2.56,23.73,1.35,1.82,433.15,48942.71,451.50,244243.67
e=5,0.75,0.00,0.69,1.60,0.25,0.66,0.94,0.00,0.94,0.94,...,0.75,15.90,8.17,16.95,6.03,8.10,2465.76,281326.68,3528.38,1402042.41
e=6,0.00,0.00,0.00,0.19,0.00,0.04,0.00,0.00,0.00,0.94,...,0.19,16.73,0.00,18.90,1.58,1.81,455.42,54840.29,167.74,274321.37
e=7,0.22,0.22,0.22,0.04,0.22,0.18,0.94,0.94,0.94,0.94,...,0.94,13.33,6.57,14.21,2.42,2.77,746.68,89526.25,781.71,446814.51


In [41]:
print(min_welfare)

np.sum(obj)

3671825.6448050328


7268675.496416553

In [ ]:
# # e = 0
# # t = 4

# e = 4
# t = 3

# print("travel_times[(e, 0, t)]:\n", travel_times[(e, 0, t)])
# print()
# print("travel_times[(e, 0, t)] + argmin_tau[(e, t)] / np.max(VoT_array[:, in_indices, t]):\n", \
#       travel_times[(e, 0, t)] + argmin_tau[(e, t)] / np.max(VoT_array[:, in_indices, t]) )
# print()
# print("travel_times[(e, 1, t)]:\n", travel_times[(e, 1, t)])
# print()
# print("Total ineligible user express lane flow at (e, t):\n", argmin_y_in_el_total[(e, 0, t, "in")])

# VoT_array[:, in_indices, t]
# # demand_array[:, in_indices]

# # print("")



In [ ]:
# obj_R

# for e in range(num_edges):
#     for t in range(T):
#         print(sum(argmin_y[(od, g, e, 0, t)] * argmin_tau[e, t] \
#                         for od in edge_to_od_dict[e] for g in in_indices) \
#                     + sum(argmin_y[(od, g, e, 1, t)] * argmin_tau[e, t] \
#                         for od in edge_to_od_dict[e] for g in el_indices) )

# VoT_array[:, :, 0]

In [ ]:
random_string = ""
for idx_rand_str in range(10):
    random_string += str(np.random.randint(1, 9))

directory_to_save = "../data/opt_tolls_subsidies_metrics/"
random_filename = "opt_DBCP___" + random_string + '.csv'

df_opt_save.to_csv(directory_to_save + random_filename)

## Test:

## <font color='red'>Colored Font Titles</font> 

# Scratch Work:

In [ ]:
x = cp.Variable(2)
y = cp.Variable(2)
v_fixed = np.array([0, 1])
objective = cp.Minimize(cp.sum_squares(x - y) + cp.sum_squares(x - v_fixed))
constraints = []
prob = cp.Problem(objective, constraints)

# The optimal objective value is returned by `prob.solve()`.
result = prob.solve()
# The optimal value for x is stored in `x.value`.
print("x.value:", x.value)
print("y.value:", y.value)
print()


## CVXPY can handle 4d arrays:

In [ ]:

I, J, K, L = 2, 3, 4, 5

# Variables:
x_test = {}
for i in range(I):
    for j in range(J):
        for k in range(K):
            for ell in range(L):
                x_test[(i, j, k, ell)] = cp.Variable(1)
            
# Objective:
func = 0.0
func += cp.sum([x_test[(i, j, k, ell)]**2 for i in range(I) for j in range(J) \
                for k in range(K) for ell in range(L)])
            
objective = cp.Minimize(func)

# Constraints:
constraints = []

for i in range(I):
    for j in range(J):
        for k in range(K):
            for ell in range(L):
                constraints += [cp.sum([x_test[(i, j, k, ell)] for i in range(I) for j in range(J) \
                                        for k in range(K) for ell in range(L) ]) == 1.0]
                constraints += [x_test[(i, j, k, ell)] >= 0.0 for i in range(I) for j in range(J) \
                                        for k in range(K) for ell in range(L)]

# Solve problem:
prob = cp.Problem(objective, constraints)
result = prob.solve()

# Print solution:
for i in range(I):
    for j in range(J):
        for k in range(K):
            for ell in range(L):
                print("i, j, k, ell:", i, j, k, ell)
                print("x_test[(i,j,k, ell)].value:", x_test[(i, j, k, ell)].value)


## Linear Approximation for Latency Function:

In [ ]:
# Variables:
v = cp.Variable(1)
            
# Objective:
func = v - 1 + cp.square(cp.maximum(v-1, 0))
objective = cp.Minimize(func)

# Constraints:
constraints = [-3.0 <= v, v <= 3.0]

# Solve problem:
prob = cp.Problem(objective, constraints)
result = prob.solve()

# Print solution:
print("v.value:", v.value)
